In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import yaml

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score


os.chdir('/home/LULAB/wboohar/CODEX/data_processing/code')
from codex_project import codex_project
from quantcell import quantcell_project, interpolate_PRC
warnings.filterwarnings('ignore')

In [ ]:
annotations_path = '/store/Projects/wboohar/PhenoCycler/annotation_strategies/marker_combos_062525_updated_verified.json'   
base_dir = '/store/Projects/wboohar/PhenoCycler' 
project_name = 'QuantCellPaper'
project_path = f'{base_dir}/{project_name}'

In [ ]:
project = quantcell_project()
project.initialize(base_path=base_dir, project_name=project_name)

In [ ]:
project.process_data()

### Create data for Fig 2A-B

In [ ]:

names = [
    'Nearest Neighbors',
    'Multilayer Perceptron',
    'Random Forest',
    'Extra Trees',
    'Decision Tree',
    'Gaussian Naive-Bayes',
    'Ridge Regression',
    "Linear SVC"
]

classifiers = [
    KNeighborsClassifier(n_jobs=-1),
    MLPClassifier(),
    RandomForestClassifier(n_jobs=-1),
    ExtraTreesClassifier(n_jobs=-1),
    DecisionTreeClassifier(),
    GaussianNB(),
    RidgeClassifier(),
    OneVsRestClassifier(LinearSVC(), n_jobs=-1)
]

In [ ]:
project.test_model_list(classifiers, names)

In [ ]:

os.makedirs(f'{base_dir}/{project_name}/base_models/', exist_ok=True)

base_clf_performance_dict={}
    
for _name in names:
    y_test = joblib.load(f'{base_dir}/{project_name}/base_models/{_name}_y_test.joblib')
    y_pred = joblib.load(f'{base_dir}/{project_name}/base_models/{_name}_y_pred.joblib')
    y_proba = joblib.load(f'{base_dir}/{project_name}/base_models/{_name}_y_proba.joblib')

    macro_AP_list = []
    weighted_AP_list = []
    macro_interp_PRC_list = []
    weighted_interp_PRC_list = []

    for fold in range(10):
        

        macro_AP = average_precision_score(y_test[fold], y_proba[fold], average='macro')
        weighted_AP = average_precision_score(y_test[fold], y_proba[fold], average='weighted')

        macro_interp_PRC = interpolate_PRC(y_test[fold], y_pred[fold], y_proba[fold], average='macro')
        weighted_interp_PRC = interpolate_PRC(y_test[fold], y_pred[fold], y_proba[fold], average='weighted')

        macro_AP_list.append(macro_AP)
        weighted_AP_list.append(weighted_AP)
        
        macro_interp_PRC_list.append(macro_interp_PRC)
        weighted_interp_PRC_list.append(weighted_interp_PRC)

    all_y_test = np.concatenate([y_test[x] for x in range(10)])
    num_classes = len(np.unique(all_y_test))

    base_clf_performance_dict[_name] = [macro_AP_list, weighted_AP_list, macro_interp_PRC_list, weighted_interp_PRC_list, num_classes]

joblib.dump(base_clf_performance_dict, f'{base_dir}/{project_name}/base_models/base_clf_performance_dict.joblib')

2C

In [ ]:
parameter_space_MLP = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (300,), (400,), (500,), (100,50), (200,50), (50, 50, 50), (50, 100, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'alpha': [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

parameter_space_trees= {
    'n_estimators': [50, 100, 200, 400],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 40, 60, 80, None],
    'bootstrap': [False, True],
    'class_weight': [None, 'balanced', 'balanced_subsample'],
}


In [ ]:
for label in ['MLPClassifier', 'RandomForestClassifier', 'ExtraTreesClassifier']:
    print(joblib.load(f'{base_dir}/{project_name}/model_selection/{label}_best_params_macro.joblib'))

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, average_precision_score

os.makedirs(f'{project_path}/model_selection/', exist_ok=True)

scorer = make_scorer(average_precision_score, 
                     average='macro',
                     response_method='predict_proba')

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=project.random_seed)

for top_base_clf in [RandomForestClassifier(), ExtraTreesClassifier(), MLPClassifier()]:
    label= top_base_clf.__class__.__name__
    if label == 'MLPClassifier':
        parameter_space = parameter_space_MLP
    else:
        parameter_space = parameter_space_trees
    clf = HalvingGridSearchCV(top_base_clf, 
                          parameter_space, 
                          n_jobs=-1, 
                          cv=cv, 
                          verbose=True, 
                          random_state=project.random_seed,
                          scoring=scorer)
    clf.fit(project.X_train, project.y_train)
    print(clf.best_params_)


    pd.DataFrame(clf.cv_results_).to_csv(f'{project_path}/model_selection/{label}_cv_results_macro.csv', index=False)
    joblib.dump(clf.best_params_, f'{project_path}/model_selection/{label}_best_params_macro.joblib')

In [ ]:
results={}

os.makedirs(f'{project_path}/model_selection/', exist_ok=True)

for top_base_clf in [RandomForestClassifier(n_jobs=-1), ExtraTreesClassifier(n_jobs=-1), MLPClassifier()]:    
    label = top_base_clf.__class__.__name__
    clf = top_base_clf
    project.set_clf(clf)
    y_test, y_pred, y_proba = project.cv_fit_pred(name=label)
    results[label+'_untuned'] = [y_test, y_pred, y_proba]

    best_params = joblib.load(f'{project_path}/model_selection/{label}_best_params_macro.joblib')
    print(f'Best parameters for {label}: {best_params}')
    if label != 'MLPClassifier':
        best_params['n_jobs'] = -1
    clf = top_base_clf.set_params(**best_params)
    project.set_clf(clf)
    y_test, y_pred, y_proba = project.cv_fit_pred(name=label+'_tuned')
    results[label+'_tuned'] = [y_test, y_pred, y_proba]
    joblib.dump(results, f'{project_path}/model_selection/top3models_results.joblib')

2D

In [ ]:
stats_loc_dict={}
best_params = joblib.load(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib')
clf = RandomForestClassifier(**best_params, n_jobs=-1)
project.set_clf(clf)

location_order = ['Membrane', 'Cytoplasm', 'Nucleus', 'Cell', 'All']
stat_order = ['Min', 'Max', 'Std.Dev.', 'Median', 'Mean', 'All']

for location in location_order:
    stats_loc_dict[location] = {}
    included_cols = [col for col in project.column_names if location in col]
    if location == 'All':
        included_cols = project.column_names
    for stat in stat_order:
        if stat == 'All':
            selected_cols = included_cols
        else:
            selected_cols = [col for col in included_cols if stat in col]

        y_test, y_pred, y_proba = project.cv_fit_pred(name=f'RF_{location}_{stat}', included_cols=selected_cols)
        stats_loc_dict[location][stat] = [y_test, y_pred, y_proba]
        joblib.dump(stats_loc_dict, f'{project_path}/model_selection/stats_loc_dict_included.joblib')

2E

In [ ]:
from sklearn.ensemble import RandomForestClassifier
stats_loc_dict={}

best_params = joblib.load(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib')
clf = RandomForestClassifier(**best_params, n_jobs=-1)
project.set_clf(clf)

location_order = ['Membrane', 'Cytoplasm', 'Nucleus', 'Cell', 'All']
stat_order = ['Min', 'Max', 'Std.Dev.', 'Median', 'Mean', 'All']

for location in location_order:
    stats_loc_dict[location] = {}
    included_cols = [col for col in project.column_names if location in col]
    if location == 'All':
        included_cols = project.column_names
    for stat in stat_order:
        if stat == 'All':
            selected_cols = included_cols
        else:
            selected_cols = [col for col in included_cols if stat in col]

        if location == 'All' and stat == 'All':
            stats_loc_dict[location][stat] = [None, None, None]
        else:
            y_test, y_pred, y_proba = project.cv_fit_pred(name=f'RF_{location}_{stat}', excluded_cols=selected_cols)
            stats_loc_dict[location][stat] = [y_test, y_pred, y_proba]

        joblib.dump(stats_loc_dict, f'{project_path}/model_selection/stats_loc_dict_excluded.joblib')

### 3A

In [ ]:
best_params = joblib.load(f'/store/Projects/wboohar/PhenoCycler/QuantCellPaper/model_selection/RandomForestClassifier_best_params_macro.joblib')

classifier = RandomForestClassifier(
    **best_params,
    n_jobs=-1,
    random_state=project.random_seed
)

project.set_clf(classifier)

project.fit()

In [ ]:
counts = project.codex.cell_type.value_counts()
fraction_annotated = {}
fraction_annotated['Conventional'] = counts[counts.index!='Other'].sum() / counts.sum()
FDR_tests = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0]

for fdr in FDR_tests:
    project.relabel_unannotated(FDR_cutoff=fdr)
    relabeled_counts = pd.Series(project.relabeled_labels).value_counts()
    fraction_annotated[f'FDR < {int(fdr*100)}%'] = relabeled_counts[relabeled_counts.index!='Other'].sum() / counts.sum()

joblib.dump(fraction_annotated, f'{project_path}/fraction_annotated.joblib')

3E and S2B

In [ ]:
encoder = project.target_encoder
if not os.path.exists(f'{project_path}/label_encoder.joblib'):
    joblib.dump(encoder, f'{project_path}/label_encoder.joblib')
else:
    encoder = joblib.load(f'{project_path}/label_encoder.joblib')

In [ ]:
codex = codex_project()
codex.read_csv(f'{project_path}/codex_conventional_{project_name}.csv', project_name=project_name)
codex.read_annotation_strategy(annotations_path)
combos=codex._marker_combos

codex.codex = codex.codex.loc[codex.codex.loc[:, 'cell_type'] != 'Other', :]
codex.codex = codex.codex.reset_index(drop=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(codex.codex, codex.codex['cell_type'], 
    test_size=0.2, 
    random_state=project.random_seed, 
    stratify=codex.codex['cell_type'])

if not os.path.exists(f'{project_path}/ComparisonsTrainTest.joblib'):
    joblib.dump((X_train, X_test, y_train, y_test), f'{project_path}/ComparisonsTrainTest.joblib')
else:
    X_train, X_test, y_train, y_test = joblib.load(f'{project_path}/ComparisonsTrainTest.joblib')
    # Reset index for model and validation sets

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

## AnnoSpat

### Marker Combos

In [ ]:
annospat_combos = pd.DataFrame(columns=combos.keys())
for ct in annospat_combos:
    n=0
    for marker in combos[ct]:
        if marker.count('+'):
            marker = marker.split('+')[0]
        
        annospat_combos.loc[n, ct] = marker
        n+=1

In [ ]:
os.makedirs(f'{project_path}/AnnoSpat', exist_ok=True)
annospat_combos.index = range(1, len(annospat_combos) + 1)
annospat_combos.to_csv(f'{project_path}/AnnoSpat/marker_combos.csv', index=True)

### Data

In [ ]:
columns=[x for x in codex.codex if ": Cell: Mean" in x]
annospat_codex = codex.codex.copy()
annospat_codex = annospat_codex.loc[:, columns]
annospat_codex = annospat_codex.reset_index(drop=True)
annospat_codex = annospat_codex.rename(columns=lambda x: x.split(':')[0])
annospat_codex.loc[:, 'ROI'] = annospat_codex.index
if not os.path.exists(f'{project_path}/AnnoSpat/marker_data.csv'):
    annospat_codex.to_csv(f'{project_path}/AnnoSpat/marker_data.csv', index=True)
else:
    annospat_codex = pd.read_csv(f'{project_path}/AnnoSpat/marker_data.csv', index_col=0)

if not os.path.exists(f'{project_path}/AnnoSpat/annospat_true_labels.csv'):
    codex.codex.loc[:, 'cell_type'].to_csv(f'{project_path}/AnnoSpat/annospat_true_labels.csv', index=True)

## Astir

### Marker Combos

In [ ]:
os.makedirs(f'{project_path}/Astir', exist_ok=True)

astir_combos = {}
astir_combos['cell_types'] = {}
for ct in combos:
    if ct not in astir_combos['cell_types']:
        astir_combos['cell_types'][ct] = []
    for marker in combos[ct]:
        if marker.count('+'):
            marker = marker.split('+')[0]
            astir_combos['cell_types'][ct].append(marker)


yaml_string = yaml.dump(astir_combos)

with open(f'{project_path}/Astir/marker_combos.yaml', 'w') as f:
    f.write(yaml_string)

### Data

In [ ]:
columns= [x for x in codex.codex if ": Cell: Mean" in x]
astir_codex = codex.codex.copy()
astir_codex.loc[:, 'index'] = astir_codex.index
astir_codex = astir_codex.loc[:, ['index'] + columns]
astir_codex.columns = astir_codex.columns.str.split(':').str[0]
if not os.path.exists(f'{project_path}/Astir/astir_true_labels.csv'):
    codex.codex.loc[:, 'cell_type'].to_csv(f'{project_path}/Astir/astir_true_labels.csv', index=False)

In [ ]:
if not os.path.exists(f'{project_path}/Astir/marker_data.csv'):
    astir_codex.to_csv(f'{project_path}/Astir/marker_data.csv', index=True)
else:
    astir_codex = pd.read_csv(f'{project_path}/Astir/marker_data.csv', index_col=0)  

## MAPS


In [ ]:
column_mask = [project._regex_marker_cols(x) for x in X_train.columns]
columns = list(X_train.columns[column_mask])
columns = [x for x in columns if 'Cell: Mean' in x]

columns.append('Cell: Area µm^2')
columns.append('cell_type')

maps_codex = X_train.copy()
maps_codex = maps_codex.loc[:, columns]
maps_codex = maps_codex.reset_index(drop=True)
maps_codex = maps_codex.rename(columns=lambda x: x.split(':')[0])

maps_codex.rename(columns={'Cell: Area µm^2': 'cellSize', 'cell_type' : 'cell_label'}, inplace=True)

os.makedirs(f'{project_path}/MAPS/data/cell_phenotyping/', exist_ok=True)


In [ ]:
maps_codex.loc[:, 'cell_label'] = encoder.transform(maps_codex.loc[:, 'cell_label'])

X = maps_codex.drop(columns=['cell_label'])
y = maps_codex['cell_label']

maps_test = X_test.loc[:, columns]
maps_test.rename(columns={'Cell: Area µm^2': 'cellSize', 'cell_type' : 'cell_label'}, inplace=True)
maps_test.loc[:, 'cell_label'] = encoder.transform(maps_test.loc[:, 'cell_label'])

maps_X_train, maps_X_valid, maps_y_train, maps_y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
maps_train = pd.concat([maps_X_train.reset_index(drop=True), maps_y_train.reset_index(drop=True)], axis=1)
maps_train.to_csv(f'{project_path}/MAPS/data/cell_phenotyping/train_features.csv', index=False)

maps_valid = pd.concat([maps_X_valid.reset_index(drop=True), maps_y_valid.reset_index(drop=True)], axis=1)  
maps_valid.to_csv(f'{project_path}/MAPS/data/cell_phenotyping/valid_features.csv', index=False)

maps_test.to_csv(f'{project_path}/MAPS/data/cell_phenotyping/test_features.csv', index=False)

## QuantCell

In [ ]:
from quantcell import quantcell_project

project = quantcell_project()
project.initialize(base_path=base_dir, project_name=project_name)
X_valid_wiped = X_valid.copy()
X_valid_wiped.loc[:, 'cell_type'] = 'Other'
project.codex = pd.concat([X_model, X_valid_wiped], axis=0, ignore_index=True)

In [ ]:
project.process_data()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
import time
best_params = joblib.load(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib')

true_labels = X_valid.loc[:, 'cell_type']
true_labels.to_csv(f'{project_path}/quantcell_true_labels.csv', index=True)

classifier = RandomForestClassifier(**best_params, n_jobs=-1, random_state=10)

project.set_clf(classifier)

start = time.time()
project.fit()
end_fit = time.time()

In [ ]:
start_time = time.time()
project.relabel_unannotated(FDR_cutoff=0.05)
end_time = time.time()
quantcell_labels = pd.Series(project.relabeled_labels)
quantcell_labels.to_csv(f'{project_path}/quantcell_labels.csv', index=False)    
elapsed_time = end_fit - start + end_time - start_time
pd.Series(elapsed_time).to_csv(f'{project_path}/time_elapsed_quantcell.csv', index=False)

4B

In [ ]:
from sklearn.ensemble import RandomForestClassifier
gene_ko_dict={}

best_params = joblib.load(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib')
clf = RandomForestClassifier(**best_params, n_jobs=-1)
project.set_clf(clf)

gene_order = [x for x in project.column_names if ': Cell: Mean' in x] # every marker has a 'Cell: Mean' column
gene_order = [x.split(': Cell: Mean')[0] for x in gene_order]
gene_order = sorted(gene_order, key=str.lower)

for marker in gene_order:
    selected_cols = [col for col in project.column_names if marker in col]
    if len(selected_cols) == 0:
        raise ValueError(f'No columns found for marker {marker}. Check the column names in the project.')
    y_test, y_pred, y_proba = project.cv_fit_pred(name=f'RF_{marker}', excluded_cols=selected_cols)
    gene_ko_dict[marker] = [y_test, y_pred, y_proba]
    joblib.dump(gene_ko_dict, f'{project_path}/model_selection/gene_ko_dict.joblib')
   